In [1]:
from datasets.wider_global_dataset_new import build_wider_dataloader
from datasets.text_test_datasets import build_text_test_loader
from datasets.image_test_datasets import build_image_test_loader
from models.encoder import Model, MLP
from evaluators.global_evaluator import GlobalEvaluator
from evaluators.np_evaluator import NPEvaluator
from loss.loss import crossmodal_triplet_loss, cos_distance, triplet_cos_loss
from loggers.logger import Logger
from manager import build_graph_optimizer
from tqdm import tqdm_notebook as tqdm
from sklearn.neighbors import DistanceMetric

from attentions.rga_attention import RGA_attend_one_to_many_batch, RGA_attend_one_to_many
import os

import torch.nn as nn
import torch.optim as optim

from configs.args import load_arg_parser

In [2]:
parser = load_arg_parser()
cfg = parser.parse_args("")
cfg.data_root = "/data/aiyucui2/wider"
root = cfg.data_root

# data path
cfg.anno_path = os.path.join(root, cfg.anno_path)
cfg.img_dir = os.path.join(root, cfg.img_dir)
cfg.val_anno_path = os.path.join(root, cfg.val_anno_path)
cfg.val_img_dir = os.path.join(root, cfg.val_img_dir)
cfg.gt_file_fn = os.path.join(root, cfg.gt_file_fn)

# meta data path
cfg.cheap_candidate_fn = os.path.join(root, cfg.cheap_candidate_fn)
cfg.vocab_path = os.path.join(root, cfg.vocab_path)
#os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"
# sys path
cfg.model_path = os.path.join(root, cfg.model_path)
cfg.output_path = os.path.join(root, cfg.output_path)
ckpt_root = "/shared/rsaas/aiyucui2/wider_person/checkpoints/initialized" #"/shared/rsaas/aiyucui2/wider_person/checkpoints/reID"
load_exp_name = "dist_fn_cosine_imgbb_resnet18_capbb_bigru_embed_size_512_batch_96_lr_0.0001_captype_sent_img_meltlayer_8_cos_margin_0.2_np_False"
cfg.load_ckpt_fn = os.path.join(ckpt_root, "resnet18_bigru_512_match_initialized.pt")
cfg.debug = False
cfg.embed_size = 512
cfg.batch_size = 96
cfg.img_backbone_opt = "resnet18"
cfg.num_gpus = 1
cfg.cap_backbone_opt = "bigru"
cfg.dim = (384,128)
cfg.dist_fn_opt = "cosine"
cfg.np = False
cfg.img_num_cut = 6
cfg.img_num_cut = 1 if not cfg.np else cfg.img_num_cut

cfg.cap_embed_type='sent'
# exp_name
cfg.exp_name = "dist_fn_{}_imgbb_{}_capbb_{}_embed_size_{}_batch_{}_lr_{}_captype_{}".format(cfg.dist_fn_opt,
                                                                       cfg.img_backbone_opt,
                                                                       cfg.cap_backbone_opt,
                                                                       cfg.embed_size,
                                                                       cfg.batch_size,
                                                                       cfg.lr,
                                                                                         cfg.cap_embed_type)
cfg.exp_name = 'debug'
cfg.model_path = os.path.join("/shared/rsaas/aiyucui2/wider_person", cfg.model_path, cfg.exp_name)
cfg.output_path = os.path.join("/shared/rsaas/aiyucui2/wider_person", cfg.output_path, cfg.exp_name)

if not os.path.exists(cfg.model_path):
    os.mkdir(cfg.model_path)
if not os.path.exists(cfg.output_path):
    os.mkdir(cfg.output_path)
# logger
logger = Logger("test.txt") #os.path.join(cfg.output_path, cfg.exp_name+".txt"))
print(cfg.exp_name)

debug


In [3]:
# train loader
train_loader = build_wider_dataloader(cfg)
cfg.num_ids = len(train_loader.dataset.person2label.values())
# test loader (loading image and text separately)
test_text_loader = build_text_test_loader(cfg) 
test_image_loader = build_image_test_loader(cfg) 

# Evaluator
Evaluator = NPEvaluator if cfg.np else GlobalEvaluator

evaluator = Evaluator(img_loader=test_image_loader, 
                          cap_loader=test_text_loader, 
                          gt_file_path=cfg.gt_file_fn,
                          embed_size=cfg.embed_size,
                          logger=logger,
                          dist_fn_opt="euclidean")
cos_evaluator = Evaluator(img_loader=test_image_loader, 
                          cap_loader=test_text_loader, 
                          gt_file_path=cfg.gt_file_fn,
                          embed_size=cfg.embed_size,
                          logger=logger,
                          dist_fn_opt="cosine")

[ds] load annotations from /data/aiyucui2/wider/wider/train/train_anns_train.json
size of dataset: 74264


In [4]:
#ckpt_path = "/shared/rsaas/aiyucui2/wider_person/checkpoints/reID/resnet18_bigru_512/resnet18_bigru_512_id_initalized.pt"
#ckpt = torch.load(ckpt_path)
cfg.num_gpus

1

In [10]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim 
import os

import torch.nn.functional as F
from models.encoder import Model, MLP
from loss.loss import triplet_cos_loss, crossmodal_triplet_loss

from attentions.rga_attention import RGA_attend_one_to_many_batch, RGA_attend_one_to_many

        
def regional_alignment_text(fulls, parts, p2fs, dist_fn_opt):
    start_index = 0
    aligned = []
    for i, jump in enumerate(p2fs):
        curr_parts = parts[start_index:start_index + jump]
        start_index += jump
        curr_full = fulls[i:i+1]
        aligned.append(RGA_attend_one_to_many(curr_full, curr_parts, dist_fn_opt))
    return torch.cat(aligned)

def regional_alignment_image(fulls, parts, dist_fn_opt):
    return RGA_attend_one_to_many_batch(fulls, parts, dist_fn_opt)
  
    
class Manager:
    def __init__(self, args, logger):
        self.log = logger.log
        self.cfg = args
        self._init_models()
        self._init_criterion()
    
    def _init_criterion(self):
        if self.cfg.dist_fn_opt == "cosine":
            self.triplet_loss = triplet_cos_loss
        elif self.cfg.dist_fn_opt == "euclidean":
            self.triplet_loss = nn.TripletMarginLoss()
        self.cls_loss = nn.CrossEntropyLoss()
        self.log("[Trainer][init] criterion initialized.")

    def _init_models(self):
        # encoder
        self.model = Model(embed_size=self.cfg.embed_size, 
                          image_opt=self.cfg.img_backbone_opt, 
                          caption_opt=self.cfg.cap_backbone_opt,
                          cap_embed_type=self.cfg.cap_embed_type,
                          img_num_cut=self.cfg.img_num_cut,
                          regional_embed_size=self.cfg.regional_embed_size).cuda()
        # id classifer
        self.id_cls = nn.Linear(self.cfg.embed_size, self.cfg.num_ids).cuda()
        # RGA image mlp
        self.rga_img_mlp = MLP(self.cfg.regional_embed_size, self.cfg.embed_size).cuda()
        # RGA text mlp
        self.rga_cap_mlp = MLP(self.cfg.embed_size, self.cfg.embed_size).cuda()
        # gpu
        self.all_models = {
            "model": self.model,
            "id_cls": self.id_cls, 
            "rga_img_mlp": self.rga_img_mlp,
            "rga_cap_mlp": self.rga_cap_mlp,
        }
        print(self.cfg.num_gpus)
        if self.cfg.num_gpus > 1:
            print('here')
            for name in self.all_models.keys():
                print('data parallel')
                self.all_models[name] = nn.DataParallel(self.all_models[name])
        # load ckpt
        self.reset_ckpt()
        
        
        self.log("[Trainer][init] model initialized.")

    def reset_ckpt(self):
        self.start_epoch = 0
        self.acc_history = []
        self.best_acc = ({'top-1':0, 'top-1': 0, 'top-1': 0}, self.start_epoch)
        if self.cfg.load_ckpt_fn == "0":
            self.log("[Trainer][init] initialize fresh model.")
            return
        ckpt = torch.load(self.cfg.load_ckpt_fn)
        self.start_epoch = ckpt["epoch"] + 1
        self.acc_history = ckpt["acc_history"]
        for name, network in self.all_models.items():
            if name in ckpt:
                network.load_state_dict(ckpt[name], strict=False)
                self.log("[Trainer][init] load pre-trained %s from %s." % (name, self.cfg.load_ckpt_fn))

              
    def save_ckpt(self, epoch, acc, fn):
        # update acc history 
        self.acc_history.append((acc, epoch))
        if acc['top-1'] > self.best_acc[0]['top-1']:
            self.best_acc = (acc, epoch)
        # ckpt 
        ckpt = {
            "epoch": epoch,
            "acc_history": self.acc_history,
            "best_acc": self.best_acc,
            }
        for name, network in self.all_models.items():
            ckpt[name] = network.module.state_dict() if isinstance(network, nn.DataParallel) else network.state_dict()

        path = os.path.join(self.cfg.model_path, fn)
        torch.save(ckpt, path)
            
    def todevice(self, batch):
        ret = []
        for arg in batch:
            if isinstance(arg, torch.Tensor):
                arg = arg.cuda()
            ret.append(arg)
        return tuple(ret)
    
    def melt_img_layer(self, num_layer_to_melt=1):
        if isinstance(self.model, nn.DataParallel):
            self.model.module.img_backbone.melt_layer(8 - num_layer_to_melt)
        else:
            self.model.img_backbone.melt_layer(8 - num_layer_to_melt)
     
    def train_epoch_global(self, train_data, optimizer, epoch, note="train"):
        self.model.train()
        cum_tri_loss, cum_id_loss = 0.0, 0.0
        for i, data in tqdm(enumerate(train_data), "%s, epoch%d" % (note, epoch)):
            # load data
            data = self.todevice(data)
            img, cap, pid = data
            
            # encode
            #img, pos_img, neg_img = self.model(img), self.model(pos_img), self.model(neg_img)
            #cap, pos_cap, neg_cap = self.model(cap), self.model(pos_cap), self.model(neg_cap)
            img, cap = self.model(img), self.model(cap)


            # loss
            tri_loss = triplet_cos_loss(img, cap, pid)
            id_loss = self.cls_loss(self.id_cls(img), pid) + self.cls_loss(self.id_cls(cap), pid)
            loss = tri_loss + id_loss

            # backpropagation
            optimizer.zero_grad(); loss.backward(); optimizer.step()
            # log
            cum_tri_loss += tri_loss.item()
            cum_id_loss += id_loss.item()
            if (i+1) % self.cfg.print_freq == 0:
                out_string = "[ep-%d, bs-%d] " % (epoch, i)
                out_string += "[tri-loss] %.6f, " % (cum_tri_loss / self.cfg.print_freq)
                out_string += "[id-loss] %.6f, " % (cum_id_loss / self.cfg.print_freq)
                self.log(out_string)
                cum_tri_loss, cum_id_loss = 0.0, 0.0
                
    def train_epoch_regional(self, train_data, optimizer, epoch, note="train"):
        self.model.train(); self.rga_img_mlp.train(); self.rga_cap_mlp.train()

        cum_tri_loss, cum_tri_image_regional_loss, cum_tri_text_regional_loss, cum_id_loss = 0.0, 0.0, 0.0, 0.0
        for i, data in tqdm(enumerate(train_data), "%s, epoch%d" % (note,epoch)):
            # load data
            data = self.todevice(data)
            (img, pos_img, neg_img, 
             cap, pos_cap, neg_cap,
             nps, pos_nps, neg_nps,
             n2c, pos_n2c, neg_n2c,
             pid, pos_pid, neg_pid) = data


            img, img_part = self.model(img)
            pos_img, pos_img_part = self.model(pos_img)
            neg_img, neg_img_part = self.model(neg_img)
            cap, pos_cap, neg_cap = self.model(cap), self.model(pos_cap), self.model(neg_cap)
            
            # N, M, T = nps.size()
            nps = self.rga_cap_mlp(self.model(nps))
            pos_nps = self.rga_cap_mlp(self.model(pos_nps))
            neg_nps = self.rga_cap_mlp(self.model(neg_nps))
            #nps = self.rga_cap_mlp(self.model(nps.reshape(-1, T))).reshape(N, M, -1)
            #pos_nps = self.rga_cap_mlp(self.model(pos_nps.reshape(-1, T))).reshape(N, M, -1)
            #neg_nps = self.rga_cap_mlp(self.model(neg_nps.reshape(-1, T))).reshape(N, M, -1)
            
            # part
            img_part = self.rga_img_mlp(img_part)
            pos_img_part = self.rga_img_mlp(pos_img_part)
            neg_img_part = self.rga_img_mlp(neg_img_part)

            img_part = RGA_attend_one_to_many_batch(cap, img_part, self.cfg.dist_fn_opt)
            pos_img_part = RGA_attend_one_to_many_batch(pos_cap, pos_img_part, self.cfg.dist_fn_opt)
            neg_img_part = RGA_attend_one_to_many_batch(neg_cap, neg_img_part, self.cfg.dist_fn_opt)
            cap_part = regional_alignment_text(img, nps, n2c, self.cfg.dist_fn_opt)
            pos_cap_part = regional_alignment_text(pos_img, pos_nps, pos_n2c, self.cfg.dist_fn_opt)
            neg_cap_part = regional_alignment_text(neg_img, neg_nps, neg_n2c, self.cfg.dist_fn_opt)
            #cap_part = RGA_attend_one_to_many_batch(img, nps, self.cfg.dist_fn_opt)
            #pos_cap_part = RGA_attend_one_to_many_batch(pos_img, pos_nps, self.cfg.dist_fn_opt)
            #neg_cap_part = RGA_attend_one_to_many_batch(neg_img, neg_nps, self.cfg.dist_fn_opt)

            # loss
            tri_loss =  crossmodal_triplet_loss(img,pos_img,neg_img, 
                                                  cap, pos_cap, neg_cap, 
                                                  self.triplet_loss, self.cfg.dist_fn_opt) 
            tri_image_regional_loss =  crossmodal_triplet_loss(img_part,pos_img_part,neg_img_part, 
                                                  cap, pos_cap, neg_cap, 
                                                  self.triplet_loss, self.cfg.dist_fn_opt) 
            tri_text_regional_loss =  crossmodal_triplet_loss(img,pos_img,neg_img, 
                                                  cap_part, pos_cap_part, neg_cap_part, 
                                                  self.triplet_loss, self.cfg.dist_fn_opt) 
            id_loss = self.cls_loss(self.id_cls(img), pid) +  self.cls_loss(self.id_cls(cap), pid)


            loss = tri_loss + tri_image_regional_loss  + tri_text_regional_loss + id_loss

            # backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            # log
            cum_tri_loss += tri_loss.item()
            cum_tri_image_regional_loss += tri_image_regional_loss.item()
            cum_tri_text_regional_loss += tri_text_regional_loss.item()
            cum_id_loss += id_loss.item()
            
            if (i+1) % self.cfg.print_freq == 0:
                out_string = "[ep-%d, bs-%d] " % (epoch, i)
                out_string += "[id-loss] %.6f, " % (cum_id_loss / self.cfg.print_freq)
                out_string += "[tri-loss] %.6f, " % (cum_tri_loss / self.cfg.print_freq)
                out_string += "[img_rga] %.6f, " %  (cum_tri_image_regional_loss / self.cfg.print_freq)
                out_string += "[cap_rga] %.6f " % (cum_tri_text_regional_loss / self.cfg.print_freq)
                self.log(out_string)
                cum_tri_loss, cum_tri_image_regional_loss, cum_tri_text_regional_loss, cum_id_loss = 0.0, 0.0, 0.0, 0.0       
    def train_epoch_id(self, train_data, optimizer, epoch, note="train"):
        self.model.train()
        self.id_cls.train()
        cum_loss = 0.0
        for i, data in tqdm(enumerate(train_data), "%s, epoch%d" % (note,epoch)):
            # load data
            data = self.todevice(data)
            (img, cap, pid) = data
            img = self.model(img)
            cap = self.model(cap)

            # loss
            loss = 0.0
            loss = loss + self.cls_loss(self.id_cls(img), pid) +  self.cls_loss(self.id_cls(cap), pid)
            cum_loss += loss.item()
            
            # backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            

            # log
            if (i+1) % self.cfg.print_freq == 0:
                self.log("ep-%d, bs-%d, [id-loss] %.6f" % (epoch, i, cum_loss / self.cfg.print_freq))
                cum_loss = 0.0

In [11]:
manager = Manager(cfg, logger)

1
[Trainer][init] load pre-trained model from /shared/rsaas/aiyucui2/wider_person/checkpoints/initialized/resnet18_bigru_512_match_initialized.pt.
[Trainer][init] load pre-trained id_cls from /shared/rsaas/aiyucui2/wider_person/checkpoints/initialized/resnet18_bigru_512_match_initialized.pt.
[Trainer][init] load pre-trained rga_img_mlp from /shared/rsaas/aiyucui2/wider_person/checkpoints/initialized/resnet18_bigru_512_match_initialized.pt.
[Trainer][init] load pre-trained rga_cap_mlp from /shared/rsaas/aiyucui2/wider_person/checkpoints/initialized/resnet18_bigru_512_match_initialized.pt.
[Trainer][init] model initialized.
[Trainer][init] criterion initialized.


In [14]:
if True:
    if cfg.np:
        acc = cos_evaluator.evaluate(manager.model, manager.rga_img_mlp, manager.rga_cap_mlp)
    else:
        acc = cos_evaluator.evaluate(manager.model)
    logger.log('[cosine   ][global] R@1: %.4f | R@5: %.4f | R@10: %.4f' % (acc['top-1'], acc['top-5'], acc['top-10']))



build db global imgs: 0it [00:00, ?it/s]
build db global imgs: 1it [00:00,  1.61it/s]
build db global imgs: 2it [00:00,  2.02it/s]
build db global imgs: 4it [00:00,  2.71it/s]
build db global imgs: 6it [00:01,  3.62it/s]
build db global imgs: 8it [00:01,  4.79it/s]
build db global imgs: 10it [00:01,  6.15it/s]
build db global imgs: 12it [00:01,  7.47it/s]
build db global imgs: 14it [00:01,  8.86it/s]
build db global imgs: 16it [00:01, 10.21it/s]
build db global imgs: 19it [00:01, 12.01it/s]
build db global imgs: 21it [00:01, 13.24it/s]
build db global imgs: 24it [00:02, 14.89it/s]
build db global imgs: 27it [00:02, 16.29it/s]
build db global imgs: 30it [00:02, 17.57it/s]
build db global imgs: 33it [00:02, 18.77it/s]
build db global caps: 0it [00:00, ?it/s]
build db global caps: 1it [00:00,  6.58it/s]
build db global caps: 14it [00:00,  9.20it/s]
build db global caps: 29it [00:00, 12.80it/s]
build db global caps: 45it [00:00, 17.67it/s]
build db global caps: 64it [00:00, 24.25it/s]
bui

[cosine   ][global] R@1: 0.3910 | R@5: 0.6418 | R@10: 0.7355


## Stage 1: ID Loss only

In [8]:

if False:
    logger.log("======== [Stage 1] ============")
    manager.melt_img_layer(num_layer_to_melt=1)
    param_to_optimize = build_graph_optimizer([manager.model, manager.id_cls])
    optimizer = optim.Adam(param_to_optimize, lr=1e-3)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10)
    
    for epoch in range(0):
        manager.train_epoch_id(train_loader, optimizer, epoch, "train-stage-1")
        acc = evaluator.evaluate(manager.model)
        logger.log('[euclidean][global] R@1: %.4f | R@5: %.4f | R@10: %.4f' % (acc['top-1'], acc['top-5'], acc['top-10']))
        acc = cos_evaluator.evaluate(manager.model)
        logger.log('[cosine   ][global] R@1: %.4f | R@5: %.4f | R@10: %.4f' % (acc['top-1'], acc['top-5'], acc['top-10']))
        scheduler.step()
        manager.save_ckpt(epoch, acc, 'stage_1_id_last.pt')
    manager.save_ckpt(epoch, acc, 'id_initialized.pt')


## Stage 2: Matching + ID Loss

In [15]:
from tqdm import tqdm_notebook as tqdm
manager.melt_img_layer(num_layer_to_melt=8)
param_to_optimize = build_graph_optimizer([manager.model, manager.id_cls])
optimizer = optim.Adam(param_to_optimize, lr=2e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5)
train_epoch = manager.train_epoch_regional if cfg.np else manager.train_epoch_global
for epoch in range(40):
    train_epoch(train_loader, optimizer, epoch, "train-stage-2")
    if cfg.np:
        acc = evaluator.evaluate(manager.model, manager.rga_img_mlp, manager.rga_cap_mlp)
        cos_acc = cos_evaluator.evaluate(manager.model, manager.rga_img_mlp, manager.rga_cap_mlp)
    else:
        acc = evaluator.evaluate(manager.model)
        cos_acc = cos_evaluator.evaluate(manager.model)
    logger.log('[euclidean][global] R@1: %.4f | R@5: %.4f | R@10: %.4f' % (acc['top-1'], acc['top-5'], acc['top-10']))
    logger.log('[cosine   ][global] R@1: %.4f | R@5: %.4f | R@10: %.4f' % (cos_acc['top-1'], cos_acc['top-5'], cos_acc['top-10']))
    scheduler.step()

[ep-0, bs-49] [tri-loss] 0.162344, [id-loss] 3.616422, 
[ep-0, bs-99] [tri-loss] 0.177356, [id-loss] 5.424442, 
[ep-0, bs-149] [tri-loss] 0.201314, [id-loss] 6.068118, 
[ep-0, bs-199] [tri-loss] 0.194129, [id-loss] 6.311744, 
[ep-0, bs-249] [tri-loss] 0.193401, [id-loss] 6.121080, 
[ep-0, bs-299] [tri-loss] 0.197741, [id-loss] 5.810260, 
[ep-0, bs-349] [tri-loss] 0.194180, [id-loss] 5.522797, 
[ep-0, bs-399] [tri-loss] 0.182322, [id-loss] 5.375436, 
[ep-0, bs-449] [tri-loss] 0.183560, [id-loss] 5.054340, 
[ep-0, bs-499] [tri-loss] 0.182816, [id-loss] 5.006310, 
[ep-0, bs-549] [tri-loss] 0.189250, [id-loss] 4.644267, 
[ep-0, bs-599] [tri-loss] 0.183038, [id-loss] 4.634500, 
[ep-0, bs-649] [tri-loss] 0.183607, [id-loss] 4.465513, 
[ep-0, bs-699] [tri-loss] 0.182451, [id-loss] 4.417586, 
[ep-0, bs-749] [tri-loss] 0.180647, [id-loss] 4.260442, 



build db global imgs: 0it [00:00, ?it/s]
build db global imgs: 1it [00:00,  1.30it/s]
build db global imgs: 3it [00:00,  1.81it/s]
build db global imgs: 5it [00:00,  2.47it/s]
build db global imgs: 7it [00:01,  3.34it/s]
build db global imgs: 9it [00:01,  4.40it/s]
build db global imgs: 11it [00:01,  5.68it/s]
build db global imgs: 13it [00:01,  7.12it/s]
build db global imgs: 15it [00:01,  8.66it/s]
build db global imgs: 17it [00:01, 10.27it/s]
build db global imgs: 19it [00:01, 11.42it/s]
build db global imgs: 22it [00:01, 13.42it/s]
build db global imgs: 25it [00:02, 15.01it/s]
build db global imgs: 28it [00:02, 16.30it/s]
build db global imgs: 31it [00:02, 17.26it/s]
build db global imgs: 33it [00:07,  4.71it/s]
build db global caps: 0it [00:00, ?it/s]
build db global caps: 1it [00:00,  8.66it/s]
build db global caps: 7it [00:00, 11.43it/s]
build db global caps: 23it [00:00, 15.84it/s]
build db global caps: 38it [00:00, 21.61it/s]
build db global caps: 51it [00:00, 28.80it/s]
buil

[euclidean][global] R@1: 0.3169 | R@5: 0.5608 | R@10: 0.6669
[cosine   ][global] R@1: 0.3881 | R@5: 0.6311 | R@10: 0.7271


[ep-1, bs-49] [tri-loss] 0.141492, [id-loss] 3.034552, 
[ep-1, bs-99] [tri-loss] 0.145947, [id-loss] 3.126549, 
[ep-1, bs-149] [tri-loss] 0.145547, [id-loss] 3.195612, 
[ep-1, bs-199] [tri-loss] 0.154682, [id-loss] 3.256456, 
[ep-1, bs-249] [tri-loss] 0.150297, [id-loss] 3.265904, 
[ep-1, bs-299] [tri-loss] 0.148756, [id-loss] 3.338955, 
[ep-1, bs-349] [tri-loss] 0.155277, [id-loss] 3.391064, 
[ep-1, bs-399] [tri-loss] 0.149029, [id-loss] 3.388000, 
[ep-1, bs-449] [tri-loss] 0.158131, [id-loss] 3.387330, 
[ep-1, bs-499] [tri-loss] 0.157327, [id-loss] 3.450862, 
[ep-1, bs-549] [tri-loss] 0.162094, [id-loss] 3.409548, 
[ep-1, bs-599] [tri-loss] 0.156192, [id-loss] 3.438409, 
[ep-1, bs-649] [tri-loss] 0.165403, [id-loss] 3.472240, 
[ep-1, bs-699] [tri-loss] 0.162495, [id-loss] 3.450916, 
[ep-1, bs-749] [tri-loss] 0.169969, [id-loss] 3.497372, 



build db global imgs: 0it [00:00, ?it/s]
build db global imgs: 1it [00:00,  1.42it/s]
build db global imgs: 4it [00:00,  1.98it/s]
build db global imgs: 6it [00:00,  2.67it/s]
build db global imgs: 8it [00:01,  3.56it/s]
build db global imgs: 10it [00:01,  4.59it/s]
build db global imgs: 12it [00:01,  5.87it/s]
build db global imgs: 14it [00:01,  7.24it/s]
build db global imgs: 16it [00:01,  8.55it/s]
build db global imgs: 18it [00:01, 10.14it/s]
build db global imgs: 20it [00:01, 11.62it/s]
build db global imgs: 23it [00:01, 13.35it/s]
build db global imgs: 26it [00:02, 14.88it/s]
build db global imgs: 29it [00:02, 16.16it/s]
build db global imgs: 32it [00:02, 17.32it/s]
build db global imgs: 33it [00:06,  4.72it/s]
build db global caps: 0it [00:00, ?it/s]
build db global caps: 1it [00:00,  8.39it/s]
build db global caps: 7it [00:00, 11.26it/s]
build db global caps: 24it [00:00, 15.50it/s]
build db global caps: 34it [00:00, 20.61it/s]
build db global caps: 46it [00:00, 26.92it/s]
bui

...

[ep-10, bs-49] [tri-loss] 0.066136, [id-loss] 0.676101, 
[ep-10, bs-99] [tri-loss] 0.069461, [id-loss] 0.706062, 
[ep-10, bs-149] [tri-loss] 0.065673, [id-loss] 0.690122, 
[ep-10, bs-199] [tri-loss] 0.066559, [id-loss] 0.678616, 
[ep-10, bs-249] [tri-loss] 0.065945, [id-loss] 0.676199, 
[ep-10, bs-299] [tri-loss] 0.062095, [id-loss] 0.672960, 
[ep-10, bs-349] [tri-loss] 0.067873, [id-loss] 0.713094, 
[ep-10, bs-399] [tri-loss] 0.069087, [id-loss] 0.701148, 
[ep-10, bs-449] [tri-loss] 0.066564, [id-loss] 0.728780, 
[ep-10, bs-499] [tri-loss] 0.064184, [id-loss] 0.705732, 
[ep-10, bs-549] [tri-loss] 0.069000, [id-loss] 0.723121, 
[ep-10, bs-599] [tri-loss] 0.062692, [id-loss] 0.698682, 
[ep-10, bs-649] [tri-loss] 0.066121, [id-loss] 0.722012, 
[ep-10, bs-699] [tri-loss] 0.064924, [id-loss] 0.727670, 
[ep-10, bs-749] [tri-loss] 0.064328, [id-loss] 0.673219, 



build db global imgs: 0it [00:00, ?it/s]
build db global imgs: 1it [00:00,  1.66it/s]
build db global imgs: 4it [00:00,  2.29it/s]
build db global imgs: 6it [00:00,  3.11it/s]
build db global imgs: 8it [00:00,  4.05it/s]
build db global imgs: 10it [00:01,  5.18it/s]
build db global imgs: 13it [00:01,  6.65it/s]
build db global imgs: 15it [00:01,  8.09it/s]
build db global imgs: 17it [00:01,  9.47it/s]
build db global imgs: 19it [00:01, 10.36it/s]
build db global imgs: 21it [00:01, 11.96it/s]
build db global imgs: 23it [00:01, 13.10it/s]
build db global imgs: 26it [00:02, 14.73it/s]
build db global imgs: 28it [00:02, 15.96it/s]
build db global imgs: 30it [00:02, 16.98it/s]
build db global imgs: 33it [00:02, 18.09it/s]
build db global caps: 0it [00:00, ?it/s]
build db global caps: 1it [00:00,  8.32it/s]
build db global caps: 17it [00:00, 11.62it/s]
build db global caps: 32it [00:00, 15.99it/s]
build db global caps: 48it [00:00, 21.82it/s]
build db global caps: 65it [00:05, 11.62it/s]
bu

[euclidean][global] R@1: 0.3334 | R@5: 0.5797 | R@10: 0.6861
[cosine   ][global] R@1: 0.4053 | R@5: 0.6428 | R@10: 0.7323


[ep-11, bs-49] [tri-loss] 0.067151, [id-loss] 0.698894, 


KeyboardInterrupt: 

In [16]:
manager.save_ckpt(epoch, acc, "match_stage2_trained.pt")